In [1]:
import string
import nltk
from bert_score import score
from ipython_genutils.py3compat import xrange
from nltk.translate.bleu_score import SmoothingFunction
from pandas import np
from tensorflow.python.keras.losses import categorical_crossentropy
import generator
from generator.main import read_data, format_data, generate_text, create_idx_to_words,  create_model, \
    read_model_weights, build_model

from generator.main import get_parameters
from nlgeval import compute_individual_metrics

c:\program files\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [2]:
parameters = get_parameters()
parameters

Parameters(model_name='model', nrows=200000, max_features=3500, max_len=20, epochs=3, start='÷', end='■')

Data Processing

In [3]:
raw_data = read_data(nrows=parameters.nrows)
raw_data


,headline
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
199995,glory on top of knights
199996,gm canola contamination report released
199997,governments work towards charlestown masterplan
199998,govt accused of using bullying tactics to force


In [4]:
X, Y, tokenizer, text = format_data(raw_data, parameters.max_features, parameters.max_len, parameters.start, parameters.end)
X, Y, tokenizer, text

(array([[   0,    0,    0, ...,    0,    1,   37],
        [   0,    0,    0, ...,    1,   37,  187],
        [   0,    0,    0, ...,   37,  187, 1250],
        ...,
        [   0,    0,    0, ...,   10,   18,   36],
        [   0,    0,    0, ...,   18,   36, 1049],
        [   0,    0,    0, ...,   36, 1049,  697]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32),
 0         ÷ aba decides against community broadcasting l...
 1         ÷ act fire witnesses must be aware of defamati...
 2         ÷ a g calls for infrastructure protection summ...
 3              ÷ air nz staff in aust strike for pay rise ■
 4         ÷ air nz strike to affect australian travellers ■
                                 ...                        
 199995                          ÷ glory on top o

In [5]:
tokenizer.word_index['community'], tokenizer.word_index[parameters.start], tokenizer.word_index[parameters.end]

(187, 1, 2)

Model Building

In [6]:
#build_model(parameters.model_name, parameters.epochs, parameters.max_features, X, Y)
model = create_model(parameters.max_features)
read_model_weights(parameters.model_name, model)
#model.evaluate(X,Y)

Text Generation

In [7]:
idx_to_words= create_idx_to_words(tokenizer)



Accuracy Calculation

In [8]:
raw_data

,headline
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
199995,glory on top of knights
199996,gm canola contamination report released
199997,governments work towards charlestown masterplan
199998,govt accused of using bullying tactics to force


In [9]:
#lol = raw_data.values.toList()
reference_data = [headline for headline in raw_data['headline']]
split_reference_data = [reference.split() for reference in reference_data]
#reference_data

In [36]:
text = generate_text(parameters.start, model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=0.75)
text 

'meets with injury'

In [48]:
cc = SmoothingFunction()
bleu1 = nltk.translate.bleu_score.sentence_bleu(split_reference_data, text.split(),weights=(9.05, 0.05, 0.05, 0.05), smoothing_function=cc.method4)
bleu1

1.215393243201835

In [32]:
text = "top kek cheburek".split()
reference = [
    "lol kek cheburek".split(),
    "lol kek uzbek".split(),
]
cc = SmoothingFunction()
bleu1 = nltk.translate.bleu_score.sentence_bleu(reference, text,weights=(1.0, 0.0, 0.0, 0.0), smoothing_function=cc.method4)
bleu2 = nltk.translate.bleu_score.sentence_bleu(reference, text,weights=(0.5, 0.5, 0, 0), smoothing_function=cc.method4)
bleu3 = nltk.translate.bleu_score.sentence_bleu(reference, text,weights=(0.33, 0.33, 0.33, 0), smoothing_function=cc.method4)
bleu4 = nltk.translate.bleu_score.sentence_bleu(reference, text,weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=cc.method4)
[bleu1, bleu2, bleu3, bleu4]

[0.6666666666666666, 0.5773502691896257, 1.2940044937315731, 2.015133275197628]

In [17]:
def generate_text_for_iteration(i, n):
    print(f'{i+1} / {n}', end='\r')
    generate_text(parameters.start, model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=0.75)
    
#generated = [ generate_text_for_iteration(i, 10) for i in range(10)]
generated = [ generate_text(parameters.start, model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=0.75) for i in range(10)]
generated[:2]

['offers new and election', 'hardie bid continues']

In [18]:
kek = []
for lol in generated[:2]:
    cheburek = nltk.translate.bleu_score.sentence_bleu(split_reference_data, lol.split())
    kek.append(cheburek)
kek

c:\program files\python\python37\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\program files\python\python37\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


[1.3478743256634661e-154, 1.254338396921439e-154]

In [13]:
def perplexity(y_true, y_pred):
    cross_entropy = categorical_crossentropy(y_true, y_pred)
    calculated_perplexity = np.power(2.0, cross_entropy)
    return calculated_perplexity

In [14]:
# metrics_dict = compute_individual_metrics(reference_data, generated[0])
# metrics_dict